# 기상 데이터 전처리

## 기상 데이터 전처리 (강수형태)


 제주도 지역별 강수형태 데이터를 가져온 후, 조건에 맞는 값을 예측값으로 지정하여 확인한 후 최종 강수형태 데이터로 저장


[데이터셋 변수 설명] 
- 일 : 일자 (1~31)
- UTC : 예측 시간(그리니치 천문대 기준 -> 실제 한국 시간으로는 +9시간)
- 예측 + 시간 : UTC 변환 이후 몇시간 뒤의 데이터를 예측할지 (EX, 7이면 7시간 이후)
- 강수 형태 : 강수 형태 (0 -> 강수 없음, 1 -> 비, 2 -> 눈, 3 -> 눈비)
    - 0의 경우 눈, 비 그 어떠한 것도 내리지 않음
    
[전처리 과정 설명] 
2:00(UTC시간대 기준)의 7시간 뒤의 예측값만을 추출하여 데이터프레임으로 구성하고자 함 
- UTC == 200, 예측 + 시간 == 7인 행만을 조건 아래 추출 
- 조건 색인 이후 불필요한 컬럼인 UTC , 예측+시간 컬럼을 drop
- 1~30의 int 형태로 나열되어 있는 일 컬럼을 날짜 형태(2021-01-01 ~ 2021-12-31)로 변환 

[기타 데이터 관련 설명] 
- 제주시 및 서귀포시 기준 행정동은 네이버 날씨의 대표값을 따름 (천지동의 경우 네이버 날씨에서는 서귀동을 기준으로 하지만 냘씨 예보는 법정동을 기준으로 하여 그에 준응하는 천지동의 데이터를 사용함) 
- 각 일자에 대한 예측값은 UTC기준 2:00AM (한국 시간, 11:00AM)에 발표된 해당 날짜의 18:00(KST)에 대한 예측값을 사용함

source : 기상청 - 기상자료개방포털 (2021/01-2021/06은 (구)단기 예보 파일셋 활용, 2021/07-2021/12은 단기 예보 파일셋 활용)
https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572

# 서귀포시 천지동 강수형태 

In [34]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

In [38]:
# 서귀포시 강수형태 데이터 불러오기
df1 = pd.read_csv('../Dataset/raw_data/weather/precip_type/precip_type_s/천지동_강수형태_202101_202106.csv') # 전반기
df2 = pd.read_csv('../Dataset/raw_data/weather/precip_type/precip_type_s/천지동_강수형태_20210701_20211231 2.csv') # 후반기

In [6]:
# 모든 데이터 칼럼명 재정렬
df1.columns = ['일','UTC','예측+시간','강수형태']
df2.columns = ['일','UTC','예측+시간','강수형태']

In [7]:
# 특정 조건에 해당하는 칼럼만 데이터로 지정
# 조건 : UTC->200, 예측+시간->7.0 (그리니치 천문대 기준 오전 2시, 즉 한국 기준 11시에서 예측한 7시간 뒤의 기온 -> 오후 6시 기온)
fc1 = df1[(df1['UTC']==200.0) & (df1['예측+시간']==7.0)] # 아라동 전반기
fc2 = df2[(df2['UTC']==200.0) & (df2['예측+시간']==7.0)] # 아라동 하반기

In [8]:
# 특정 위치에 결측치가 존재하여 이 값을 채우기
# 7월 15일 값이 결측치이므로 7월 14일 13시에 예측한 7월 15일 예측값을 사용하도록 한다.
fc2.loc[6619] = [15,200.0,7.0,0.0]

C:\Users\LeeSH\AppData\Local\Temp\ipykernel_11112\2020814971.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc2.loc[6619] = [15,200.0,7.0,0.0]


In [9]:
# 인덱스 기준으로 정렬
fc2 = fc2.sort_index()

In [10]:
# 함수를 이용해서 2021-01-01부터 2021-12-31까지 모든 데이터 불러옴
from datetime import datetime, timedelta
# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정
date_list = date_range("2021-01-01", "2021-12-31")
# 대표적으로 앞의 5개만 추출
date_list[:5]

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05']

In [11]:
# 6개월씩 나뉘어진 데이터를 하나로 합침
chunji = pd.concat([fc1,fc2])
# date_list를 서귀포시 데이터(chunji)의 날짜 칼럼으로 갖고옴
chunji['날짜'] = date_list
# 칼럼 순서 지정 및 특정 데이터만 가져옴
chunji = chunji[['날짜','일','강수형태']]
# 길이 확인(1월1일부터 12월 31일까지 이상없이 있는지)
len(chunji)

365

In [12]:
# 데이터 저장
chunji.to_csv('../Dataset/raw_data/weather/천지동강수형태.csv', index=False, encoding = 'utf-8-sig')

# 제주시 아라동 강수형태

In [14]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [15]:
from glob import glob

In [21]:
file_names = glob('../Dataset/raw_data/weather/precip_type/precip_type_j/*.csv')
precipitation_1 = pd.DataFrame() 


for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='cp949')
    temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'precipitation type'}, inplace=True)

    precipitation_1 = pd.concat([precipitation_1, temp])


precipitation_1

,day,hour,forecast,precipitation type
0,1,200.0,4.0,0.0
1,1,200.0,7.0,0.0
2,1,200.0,10.0,0.0
3,1,200.0,13.0,0.0
4,1,200.0,16.0,0.0
...,...,...,...,...
93650,31,2300.0,60.0,0.0
93651,31,2300.0,61.0,0.0
93652,31,2300.0,62.0,0.0
93653,31,2300.0,63.0,0.0


In [22]:
# 정보 확인
precipitation_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120448 entries, 0 to 93654
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   day                 120448 non-null  object 
 1   hour                120260 non-null  float64
 2   forecast            120260 non-null  float64
 3   precipitation type  120260 non-null  float64
dtypes: float64(3), object(1)
memory usage: 4.6+ MB


In [23]:
# 결측치 확인 
precipitation_1.isnull().sum()

day                     0
hour                  188
forecast              188
precipitation type    188
dtype: int64

In [24]:
# 조건 색인으로 데이터프레임 재구성
precipitation_j = precipitation_1[(precipitation_1['hour'] == 200) & (precipitation_1['forecast'] == 7)]

precipitation_j

,day,hour,forecast,precipitation type
1,1,200.0,7.0,0.0
149,2,200.0,7.0,0.0
297,3,200.0,7.0,0.0
445,4,200.0,7.0,0.0
593,5,200.0,7.0,0.0
...,...,...,...,...
91112,27,200.0,7.0,0.0
91621,28,200.0,7.0,0.0
92130,29,200.0,7.0,0.0
92639,30,200.0,7.0,2.0


In [25]:
# 누락 데이터 확인
precipitation_j['day'].value_counts()

 1     12
 2     12
 28    12
 27    12
 26    12
 25    12
 24    12
 23    12
 22    12
 21    12
 20    12
 19    12
 18    12
 17    12
 16    12
 15    12
 14    12
 13    12
 12    12
 11    12
 10    12
 9     12
 8     12
 7     12
 6     12
 5     12
 4     12
 3     12
 29    11
 30    11
 31     7
Name: day, dtype: int64

In [26]:
# 불필요한 열 제거 후 float -> int형으로 변경
precipitation_j = precipitation_j.drop(columns = {'day','hour','forecast'}).astype(int)


precipitation_j

,precipitation type
1,0
149,0
297,0
445,0
593,0
...,...
91112,0
91621,0
92130,0
92639,2


In [27]:
# 병합시킬 데이터 불러오기
jeju = pd.read_csv('../Dataset/raw_data/weather/jeju_weather.csv')

In [28]:
# 제주시 강수형태 데이터 기존 데이터와 병합
jeju = pd.concat([jeju.reset_index(drop=True), precipitation_j.reset_index(drop=True)],axis=1)

jeju

,날짜,공휴일,요일,humidity(%),강수확률,최고기온,precipitation type
0,2021-01-01,1,금,75,30,5.0,0
1,2021-01-02,0,토,75,30,4.0,0
2,2021-01-03,0,일,75,30,7.0,0
3,2021-01-04,0,월,75,0,5.0,0
4,2021-01-05,0,화,75,30,2.0,0
...,...,...,...,...,...,...,...
360,2021-12-27,0,월,80,30,6.0,0
361,2021-12-28,0,화,85,30,8.0,0
362,2021-12-29,0,수,75,30,4.0,0
363,2021-12-30,0,목,65,60,2.0,2


In [29]:
# 제주시 데이터셋 csv 저장
jeju.to_csv('../Dataset/raw_data/weather/jeju_weather.csv' ,index = False)